# IMPORTING THE MODEULES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# GETTING THE DATA

In [2]:
# getting the training data
train_data = pd.read_csv("train_data.csv")
train_data.drop(["Id","date"],axis=1,inplace=True)
train_data = train_data.values

In [3]:
# getting the training data labels
labels = pd.read_csv("train_target.csv")
labels = labels["target"].to_numpy()

# NORMALIZING THE DATA

In [4]:
from sklearn.preprocessing import MinMaxScaler
m = MinMaxScaler()
m.fit(train_data)
train_data = m.transform(train_data)

# CREATING WINDOWS AND HORIZON FROM THE DATASET

In [5]:
''' 
 we will use a window of size 7 and a horizon of size 1
 this means that we will give our model the data of last 7 sessions and expect it to predict the
 next session
'''
windows = []
horizon = []
i = 0
while(i+7<len(train_data)):
    lab = []
    lab.append(train_data[i])
    lab.append(train_data[i+1])
    lab.append(train_data[i+2])
    lab.append(train_data[i+3])
    lab.append(train_data[i+4])
    lab.append(train_data[i+5])
    lab.append(train_data[i+6])
    windows.append(lab)
    horizon.append(labels[i+7])
    i+=1
print(np.array(windows).shape)
print(np.array(horizon).shape)  
windows = np.array(windows)
horizon = np.array(horizon)

(25493, 7, 16)
(25493,)


# BUILDING THE MODEL
* IN THIS EXPERIMENT, I HAVE USED A TEMPORAL CONVOLUTIONAL NEURAL NETWORK(TCNN) TO TRAIN ON THE GIVEN DATA

In [6]:
# building the TCNN
from tensorflow.keras.layers import Input, Conv1D, Dense, Lambda, Flatten, Concatenate
from tensorflow.keras import Model
from tensorflow.keras import metrics
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import utils
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

inputs = Input(shape=(7,16))

# extract our features using a Convolutional layers, hence "CNN"
feature_extraction = Conv1D(64, 2, activation='relu')(inputs)
# split layer into three regions based on time, hence "Temporal"
long_term = Lambda( lambda x: tf.split(x, num_or_size_splits=3, axis=1)[0])(feature_extraction)
mid_term = Lambda( lambda x: tf.split(x, num_or_size_splits=3, axis=1)[1])(feature_extraction)
short_term = Lambda( lambda x: tf.split(x, num_or_size_splits=3, axis=1)[2])(feature_extraction)

long_term_conv = Conv1D(1, 1, activation='relu')(long_term)
mid_term_conv = Conv1D(1, 1, activation='relu')(mid_term)
short_term_conv = Conv1D(1, 1, activation='relu')(short_term)

# combine three layers back into one
combined = Concatenate(axis=1)([long_term_conv, mid_term_conv, short_term_conv])

# flattening is required since our input is a 2D matrix
flattened = Flatten()(combined)

x = Dense(20, activation='softmax')(flattened)
# 1 neuron with the sigmoid activation function to determine the probability of next session
outputs = Dense(1,activation="sigmoid")(x)
# specify input and output layers of our model
model = Model(inputs=inputs, outputs=outputs)

# COMPILING THE MODEL

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy", metrics="accuracy")

# FITTING THE MODEL

In [8]:
# training for the first 100 epochs
history1 = model.fit(windows, horizon, epochs=100)

Epoch 1/100
797/797 [==============================] - 18s 6ms/step - loss: 0.6930 - accuracy: 0.5102
Epoch 2/100
797/797 [==============================] - 4s 6ms/step - loss: 0.6928 - accuracy: 0.5068
Epoch 3/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6926 - accuracy: 0.5106
Epoch 4/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6927 - accuracy: 0.5097
Epoch 5/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6926 - accuracy: 0.5124
Epoch 6/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6925 - accuracy: 0.5159
Epoch 7/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6924 - accuracy: 0.5152
Epoch 8/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6924 - accuracy: 0.5172
Epoch 9/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6924 - accuracy: 0.5146
Epoch 10/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6924 - accur

797/797 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5274
Epoch 82/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5230
Epoch 83/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5251
Epoch 84/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5239
Epoch 85/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6910 - accuracy: 0.5245
Epoch 86/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6910 - accuracy: 0.5227
Epoch 87/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5259
Epoch 88/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5256
Epoch 89/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6910 - accuracy: 0.5254
Epoch 90/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6910 - accuracy: 

In [11]:
# training with a reduced learning rate for another 100 epochs
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.00001),
              loss="binary_crossentropy", 
              metrics="accuracy")
history2 = model.fit(windows, horizon, epochs=100)

Epoch 1/100
797/797 [==============================] - 5s 6ms/step - loss: 0.6903 - accuracy: 0.5290
Epoch 2/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6903 - accuracy: 0.5298
Epoch 3/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6902 - accuracy: 0.5300
Epoch 4/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6902 - accuracy: 0.5296
Epoch 5/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6902 - accuracy: 0.5295
Epoch 6/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6901 - accuracy: 0.5298
Epoch 7/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6901 - accuracy: 0.5300
Epoch 8/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6901 - accuracy: 0.5295
Epoch 9/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6901 - accuracy: 0.5299
Epoch 10/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6901 - accura

797/797 [==============================] - 4s 5ms/step - loss: 0.6899 - accuracy: 0.5300
Epoch 82/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6899 - accuracy: 0.5299
Epoch 83/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6899 - accuracy: 0.5299
Epoch 84/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6899 - accuracy: 0.5296
Epoch 85/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6899 - accuracy: 0.5303
Epoch 86/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6899 - accuracy: 0.5301
Epoch 87/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6899 - accuracy: 0.5301
Epoch 88/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6899 - accuracy: 0.5300
Epoch 89/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6899 - accuracy: 0.5305
Epoch 90/100
797/797 [==============================] - 4s 5ms/step - loss: 0.6899 - accuracy: 

In [12]:
# saving the model
model.save("TCNN.h5")

# MAKING PREDICTIONS ON THE TEST DATA

In [13]:
test_data = pd.read_csv("test_data.csv")
test_data.drop(["Id","date"],axis=1,inplace=True)
test_data = test_data.values
# normalizing the test data
from sklearn.preprocessing import MinMaxScaler
s = MinMaxScaler()
s.fit(test_data)
test_data = s.transform(test_data)

In [16]:
# splitting our test data into windows of size 7
windows_test = []
i = 0
while(i+6<len(test_data)):
    lab = []
    lab.append(test_data[i])
    lab.append(test_data[i+1])
    lab.append(test_data[i+2])
    lab.append(test_data[i+3])
    lab.append(test_data[i+4])
    lab.append(test_data[i+5])
    lab.append(test_data[i+6])
    windows_test.append(lab)
    i+=1
print(np.array(windows_test).shape)  
windows_test = np.array(windows_test)

(10311, 7, 16)


In [17]:
preds = model.predict(windows_test)
# rounding the probabilities using tf.round()
preds = tf.round(preds).numpy()
# casting the predictions to the int64 data type
preds = tf.cast(preds,dtype=tf.int64).numpy()
# reducing the dimensions using tf.squeeze()
preds = tf.squeeze(preds).numpy()
preds
'''
 for the remaining sessions(since our window size is 7 and it does not divide our test data size), 
 0 was filled in. It was done only for the remaining 6 test data instances 
 out of 10317 test data instances
'''

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

# THANK YOU 😊